# Wielowarstwowa sieć neuronowa

(*Multilayer perceptron*, *feedforward neural network*)



**Uwaga:** "Input layer" pomimo tego, że ma w nazwie słowo "warstwa", to tak naprawdę to nie jest żadna warstwa sieci... To są po prostu dane wejściowe... Niestety przyjęło się literaturze nazywanie tego w ten sposób, co jest mylące :(


Sieci uczy sie metodą spadku gradientu (pewnymi wariantami tej metody). Uczenie wykorzystuje algorytm **propagacji wstecznej** (https://en.wikipedia.org/wiki/Backpropagation).

<br>

<br>

<br>

**Uwaga!** Sieci neuronowe absolutnie zawsze wymagają zestandaryzowanych danych! Niezależnie od tego czy wykorzystujemy regularyzację czy nie i niezależnie od typu sieci!

<br>

<br>

### Fakt matematyczny: jednowarstwową siecią możemy otrzymać dowolny kształt. 

Co z tego wynika? To, że (teoretycznie) zawsze wystarczy sieć jednowarstwowa (odpowiednio duża). W praktyce rzeczywiście z reguły wystarcza jedna warstwa, ale mimo wszystko zawsze warto sprawdzić czy 2 (lub 3) nie zadziałają przypadkiem lepiej. Przy czym jeżeli dla dwóch wartsw jest gorzej, to nie ma sensu sprawdzać dla większej ilości.

In [2]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score


from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

# Zad
* Wczytaj zbiór danych
* Podziel dane na train test
* Wykonaj uczenie modeli (dobierz najlepsze parametry)
    * LogisticRegression
    * LinearSVC
    * SVC
    * KNeighborsClassifier
    * DecisionTreeClassifier
    * RandomForestClassifier
    * BaggingClassifier
    * ExtraTreesClassifier
    * AdaBoostClassifier
    * GradientBoostingClassifier
    * VotingClassifier
    * xgboost.XGBClassifier
* Porównaj wyniki na zbiorze uczącym    

In [8]:
dataset = np.loadtxt('Dane/diabetes.csv', delimiter=",", skiprows=1)

X = dataset[:,0:8]
Y = dataset[:,8]

print(X.shape)
print(np.mean(Y))

seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

from sklearn.model_selection import StratifiedKFold

seed=123
kfold = StratifiedKFold(n_splits=5, random_state=seed, shuffle=True)

(768, 8)
0.3489583333333333


In [45]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

pipe = Pipeline([('preprocessing', StandardScaler()), ('classifier', SVC(C=1,kernel='rbf',probability=True))])

param_grid = {
            'preprocessing': [StandardScaler(), None],
            'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
            'classifier__gamma': [0.001, 0.01, 0.1, 1, 10, 100]
}

grid_2 = GridSearchCV(pipe, param_grid, cv=kfold, return_train_score=True)

grid_2.fit(X_train, y_train)
grid_2.best_params_

{'classifier__C': 10,
 'classifier__gamma': 0.001,
 'preprocessing': StandardScaler()}

In [14]:
pipe = Pipeline([('preprocessing', StandardScaler()), ('classifier', LogisticRegression(C=1))])

param_grid = {
            'preprocessing': [StandardScaler(), None],
            'classifier__C': [1, 10, 100, 1000, 10000]
}

grid_3 = GridSearchCV(pipe, param_grid, cv=kfold, return_train_score=True)

grid_3.fit(X_train, y_train)
grid_3.best_params_

/Users/mateuszrajski/psi/podstawy_sztucznej_inteligencji/env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/mateuszrajski/psi/podstawy_sztucznej_inteligencji/env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:

{'classifier__C': 100, 'preprocessing': None}

In [ ]:
pipe = Pipeline([('preprocessing', StandardScaler()), ('classifier', LinearSVC(C=1))])

param_grid = {
            'preprocessing': [StandardScaler(), None],
            'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]
}

grid_1 = GridSearchCV(pipe, param_grid, cv=kfold, return_train_score=True)

grid_1.fit(X_train, y_train)
grid_1.best_params_

In [44]:
from sklearn.neighbors import KNeighborsClassifier as KNN

pipe = Pipeline([('preprocessing', StandardScaler()), ('classifier', KNN())])

param_grid = {
            'preprocessing': [StandardScaler()],
            'classifier__n_neighbors': [n for n in range(2,20)]
}

grid_4 = GridSearchCV(pipe, param_grid, cv=kfold, return_train_score=True)

grid_4.fit(X_train, y_train)
grid_4.best_params_

{'classifier__n_neighbors': 18, 'preprocessing': StandardScaler()}

In [18]:
pipe = Pipeline([('preprocessing', StandardScaler()), ('classifier', DecisionTreeClassifier())])

param_grid = {
            'preprocessing': [StandardScaler(), None],
            'classifier__max_depth': [4,5,6,7,8,9,10,11,12,15,20,30,40],
            'classifier__criterion': ['entropy', 'gini'],
            'classifier__min_samples_leaf': [2, 3, 6]
}

grid_5 = GridSearchCV(pipe, param_grid, cv=kfold, return_train_score=True)

grid_5.fit(X_train, y_train)
grid_5.best_params_

{'classifier__criterion': 'entropy',
 'classifier__max_depth': 5,
 'classifier__min_samples_leaf': 2,
 'preprocessing': StandardScaler()}

In [21]:
pipe = Pipeline([('preprocessing', StandardScaler()), ('classifier', RandomForestClassifier())])

param_grid = {
            'preprocessing': [StandardScaler(), None],
            'classifier__max_depth': [4,5,6,7,8,9,10,11],
            'classifier__min_samples_leaf': [2, 3],
            'classifier__n_estimators': [10, 20]
}

grid_6 = GridSearchCV(pipe, param_grid, cv=kfold, return_train_score=True)

grid_6.fit(X_train, y_train)
grid_6.best_params_

{'classifier__max_depth': 4,
 'classifier__min_samples_leaf': 2,
 'classifier__n_estimators': 20,
 'preprocessing': StandardScaler()}

In [22]:
from sklearn.ensemble import BaggingClassifier

pipe = Pipeline([('preprocessing', StandardScaler()), ('classifier', BaggingClassifier(DecisionTreeClassifier()))])

param_grid = {"classifier__base_estimator__max_depth": [3,20],
          "classifier__base_estimator__max_features": [None, "auto"],
          "classifier__base_estimator__min_samples_leaf": [1, 3, 10],
          'classifier__max_samples': [0.5, 1.0],
          'classifier__n_estimators': [2, 10, 20],
}

grid_7 = GridSearchCV(pipe, param_grid, cv=kfold, return_train_score=True)

grid_7.fit(X_train, y_train)
grid_7.best_params_

{'classifier__base_estimator__max_depth': 3,
 'classifier__base_estimator__max_features': 'auto',
 'classifier__base_estimator__min_samples_leaf': 3,
 'classifier__max_samples': 1.0,
 'classifier__n_estimators': 20}

In [25]:
from sklearn.ensemble import AdaBoostClassifier

param_grid = {
    'n_estimators': [10, 20, 50, 70, 200, 400],
    'learning_rate': [10**n for n in range(-5,5)]
}

grid_8 = GridSearchCV(AdaBoostClassifier(), param_grid, cv=kfold, return_train_score=True)
grid_8.fit(X_train, y_train)
grid_8.best_params_

/Users/mateuszrajski/psi/podstawy_sztucznej_inteligencji/env/lib/python3.8/site-packages/sklearn/ensemble/_weight_boosting.py:626: RuntimeWarning: overflow encountered in exp
  sample_weight *= np.exp(
/Users/mateuszrajski/psi/podstawy_sztucznej_inteligencji/env/lib/python3.8/site-packages/sklearn/ensemble/_weight_boosting.py:506: UserWarning: Sample weights have reached infinite values, at iteration 1, causing overflow. Iterations stopped. Try lowering the learning rate.
  return super().fit(X, y, sample_weight)
/Users/mateuszrajski/psi/podstawy_sztucznej_inteligencji/env/lib/python3.8/site-packages/sklearn/ensemble/_weight_boosting.py:626: RuntimeWarning: overflow encountered in exp
  sample_weight *= np.exp(
/Users/mateuszrajski/psi/podstawy_sztucznej_inteligencji/env/lib/python3.8/site-packages/sklearn/ensemble/_weight_boosting.py:506: UserWarning: Sample weights have reached infinite values, at iteration 1, causing overflow. Iterations stopped. Try lowering the learning rate.
  re

{'learning_rate': 0.1, 'n_estimators': 200}

In [26]:
from sklearn.ensemble import GradientBoostingClassifier

param_grid = {
    "max_features":["log2","sqrt"],
    'learning_rate': [10**n for n in range(-4,4)],
    'max_depth': [3,4,5],
    'min_samples_leaf': [4,5,6],
    'n_estimators': [5,10,15,20]
}

grid_9 = GridSearchCV(GradientBoostingClassifier(), param_grid, cv=kfold, return_train_score=True)
grid_9.fit(X_train, y_train)
grid_9.best_params_

{'learning_rate': 0.1,
 'max_depth': 3,
 'max_features': 'log2',
 'min_samples_leaf': 4,
 'n_estimators': 20}

In [27]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression


clf = VotingClassifier(estimators=[ 
    ('lr',LogisticRegression()),
    ('rf', RandomForestClassifier()),
    ], voting='soft')

param_grid = {
    'lr__C': [10**n for n in range(-5,5)],
    'rf__n_estimators': [5, 10, 15, 20, 50]
}

grid_10 = GridSearchCV(clf, param_grid, cv=kfold, return_train_score=True)
grid_10.fit(X_train, y_train)
grid_10.best_params_


/Users/mateuszrajski/psi/podstawy_sztucznej_inteligencji/env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/mateuszrajski/psi/podstawy_sztucznej_inteligencji/env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:

{'lr__C': 1000, 'rf__n_estimators': 5}

In [28]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats.distributions import uniform, randint

param_distribution = {
    'max_depth': randint(3, 11),
    'learning_rate': uniform(0.001, 0.1-0.001),
    'n_estimators': randint(50, 400),
    'gamma': uniform(0,2),
    'colsample_bytree': uniform(0.5, 0.5),
    'subsample': uniform(0.5, 0.5),
    'min_child_weight': randint(1, 11)
}
from xgboost import XGBClassifier

grid_11 = RandomizedSearchCV(XGBClassifier(), param_distributions=param_distribution, cv=kfold,return_train_score=True)
grid_11.fit(X_train, y_train)
grid_11.best_params_

{'colsample_bytree': 0.5433975405820906,
 'gamma': 1.0938338589890677,
 'learning_rate': 0.008114818848364836,
 'max_depth': 5,
 'min_child_weight': 9,
 'n_estimators': 164,
 'subsample': 0.7272825799628921}

In [32]:
from sklearn.ensemble import ExtraTreesClassifier

param_grid = {
    'n_estimators':[10,20,30,40,50,100],
    'max_depth': [4,5,10,20],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False],
}

grid_12 = GridSearchCV(ExtraTreesClassifier(), param_grid=param_grid, cv=kfold,return_train_score=True)
grid_12.fit(X_train, y_train)
grid_12.best_params_

{'bootstrap': True,
 'max_depth': 10,
 'max_features': 'sqrt',
 'n_estimators': 100}

In [37]:
from sklearn import  metrics


models = []
models.append(('SVM linear', grid_1.best_estimator_))
models.append(('SVM rbf', grid_2.best_estimator_))
models.append(('LR', grid_3.best_estimator_))
models.append(('KNN', grid_4.best_estimator_))
models.append(('DecisionTreeClassifier', grid_5.best_estimator_))
models.append(('BaggingClassifier', grid_7.best_estimator_))
models.append(('RandomForestClassifier', grid_6.best_estimator_))
models.append(('ExtraTreesClassifier', grid_12.best_estimator_))
models.append(('AdaBoostClassifier', grid_8.best_estimator_))
models.append(('GradientBoostingClassifier', grid_9.best_estimator_))
models.append(('XGBClassifier', grid_11.best_estimator_))
models.append(('voting_clf', grid_10.best_estimator_))

precision_score = []
recall_score = []
f1_score = []
accuracy_score = []
roc_auc_score = []
for name, model in models:
    print(name)
    print("precision_score: {}".format(metrics.precision_score(y_test , model.predict(X_test)) ))
    print("recall_score: {}".format( metrics.recall_score(y_test , model.predict(X_test)) ))
    print("f1_score: {}".format( metrics.f1_score(y_test , model.predict(X_test)) ))
    print("accuracy_score: {}".format( metrics.accuracy_score(y_test , model.predict(X_test)) ))
    
    if (name == 'SVM linear'):
        print("roc_auc_score: {}".format( metrics.roc_auc_score(y_test , model.decision_function(X_test)) ))            
    else:
        print("roc_auc_score: {}".format( metrics.roc_auc_score(y_test , model.predict_proba(X_test)[:,1]) ))
    
    precision_score.append(metrics.precision_score(y_test , model.predict(X_test)))
    recall_score.append(metrics.recall_score(y_test , model.predict(X_test)))
    f1_score.append( metrics.f1_score(y_test , model.predict(X_test)))
    accuracy_score.append(metrics.accuracy_score(y_test , model.predict(X_test)))
    if (name == 'SVM linear'):
        roc_auc_score.append(metrics.roc_auc_score(y_test , model.decision_function(X_test)))        
    else:    
        roc_auc_score.append(metrics.roc_auc_score(y_test , model.predict_proba(X_test)[:,1]))

SVM linear
precision_score: 0.7341772151898734
recall_score: 0.6304347826086957
f1_score: 0.6783625730994152
accuracy_score: 0.7834645669291339
roc_auc_score: 0.8299785292538916
SVM rbf
precision_score: 0.7605633802816901
recall_score: 0.5869565217391305
f1_score: 0.6625766871165645
accuracy_score: 0.7834645669291339
roc_auc_score: 0.8349771873322598
LR
precision_score: 0.7215189873417721
recall_score: 0.6195652173913043
f1_score: 0.6666666666666666
accuracy_score: 0.7755905511811023
roc_auc_score: 0.8271604938271605
KNN
precision_score: 0.7352941176470589
recall_score: 0.5434782608695652
f1_score: 0.625
accuracy_score: 0.7637795275590551
roc_auc_score: 0.8052871712292001
DecisionTreeClassifier
precision_score: 0.703125
recall_score: 0.4891304347826087
f1_score: 0.576923076923077
accuracy_score: 0.7401574803149606
roc_auc_score: 0.7966317767042406
BaggingClassifier
precision_score: 0.6666666666666666
recall_score: 0.5434782608695652
f1_score: 0.5988023952095807
accuracy_score: 0.736220

In [40]:
import pandas as pd
d = {'precision_score': precision_score, 
     'recall_score': recall_score, 
     'f1_score': f1_score,
     'accuracy_score' : accuracy_score,
     'roc_auc_score' : roc_auc_score
    }
df = pd.DataFrame(data=d)
df.insert(loc=0, column='Method', value=['SVM linear','SVM rbf','LR','KNN', 'DecisionTreeClassifier','BaggingClassifier','RandomForestClassifier','ExtraTreesClassifier', 'AdaBoostClassifier','GradientBoostingClassifier','XGBClassifier', 'voting'])
df

,Method,precision_score,recall_score,f1_score,accuracy_score,roc_auc_score
0,SVM linear,0.734177,0.630435,0.678363,0.783465,0.829979
1,SVM rbf,0.760563,0.586957,0.662577,0.783465,0.834977
2,LR,0.721519,0.619565,0.666667,0.775591,0.827160
3,KNN,0.735294,0.543478,0.625000,0.763780,0.805287
4,DecisionTreeClassifier,0.703125,0.489130,0.576923,0.740157,0.796632
5,BaggingClassifier,0.666667,0.543478,0.598802,0.736220,0.818170
6,RandomForestClassifier,0.720588,0.532609,0.612500,0.755906,0.830784
7,ExtraTreesClassifier,0.750000,0.521739,0.615385,0.763780,0.834071
8,AdaBoostClassifier,0.651685,0.630435,0.640884,0.744094,0.813775
9,GradientBoostingClassifier,0.680000,0.554348,0.610778,0.744094,0.821826


# MLPClassifier

Dodajmy model sieci neuronowej

In [41]:
from sklearn.neural_network import MLPClassifier

In [42]:
model = MLPClassifier((20,10))
model.fit(X_train, y_train)

y_pred = model.predict_proba(X_test)[:,1]
predictions = y_pred.round()

accuracy = metrics.accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0), "AUC: ", metrics.roc_auc_score(y_score=y_pred,y_true=y_test))

Accuracy: 68.11% AUC:  0.6843129361245304


/Users/mateuszrajski/psi/podstawy_sztucznej_inteligencji/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


# Zad
Wykonaj Walidację krzyżową

In [48]:
pipe = Pipeline([('preprocessing', StandardScaler()), ('classifier', MLPClassifier())])

param_grid = {
            'preprocessing': [StandardScaler(), None],
            'classifier__hidden_layer_sizes': [(20,10)],
            'classifier__learning_rate_init': [0.001],#, 0.01, 0.1],
            'classifier__max_iter': [100],
            'classifier__batch_size': [8, 16,32],
}

grid_13 = GridSearchCV(pipe, param_grid, cv=kfold, return_train_score=True)

grid_13.fit(X_train, y_train)
grid_13.best_params_

/Users/mateuszrajski/psi/podstawy_sztucznej_inteligencji/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/mateuszrajski/psi/podstawy_sztucznej_inteligencji/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/mateuszrajski/psi/podstawy_sztucznej_inteligencji/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/mateuszrajski/psi/podstawy_sztucznej_inteligencji/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Sto

{'classifier__batch_size': 16,
 'classifier__hidden_layer_sizes': (20, 10),
 'classifier__learning_rate_init': 0.001,
 'classifier__max_iter': 100,
 'preprocessing': StandardScaler()}

In [49]:
metrics.accuracy_score(y_test, grid_2.best_estimator_.predict(X_test))

0.7834645669291339

In [50]:
from sklearn import  metrics


models = []
models.append(('SVM linear', grid_1.best_estimator_))
models.append(('SVM rbf', grid_2.best_estimator_))
models.append(('LR', grid_3.best_estimator_))
models.append(('KNN', grid_4.best_estimator_))
models.append(('DecisionTreeClassifier', grid_5.best_estimator_))
models.append(('BaggingClassifier', grid_7.best_estimator_))
models.append(('RandomForestClassifier', grid_6.best_estimator_))
models.append(('ExtraTreesClassifier', grid_12.best_estimator_))
models.append(('AdaBoostClassifier', grid_8.best_estimator_))
models.append(('GradientBoostingClassifier', grid_9.best_estimator_))
models.append(('XGBClassifier', grid_11.best_estimator_))
models.append(('voting_clf', grid_10.best_estimator_))
models.append(('MLP', grid_13.best_estimator_))

precision_score = []
recall_score = []
f1_score = []
accuracy_score = []
roc_auc_score = []
for name, model in models:
    print(name)
    print("precision_score: {}".format(metrics.precision_score(y_test , model.predict(X_test)) ))
    print("recall_score: {}".format( metrics.recall_score(y_test , model.predict(X_test)) ))
    print("f1_score: {}".format( metrics.f1_score(y_test , model.predict(X_test)) ))
    print("accuracy_score: {}".format( metrics.accuracy_score(y_test , model.predict(X_test)) ))
    
    if (name == 'SVM linear'):
        print("roc_auc_score: {}".format( metrics.roc_auc_score(y_test , model.decision_function(X_test)) ))            
    else:
        print("roc_auc_score: {}".format( metrics.roc_auc_score(y_test , model.predict_proba(X_test)[:,1]) ))
    
    precision_score.append(metrics.precision_score(y_test , model.predict(X_test)))
    recall_score.append(metrics.recall_score(y_test , model.predict(X_test)))
    f1_score.append( metrics.f1_score(y_test , model.predict(X_test)))
    accuracy_score.append(metrics.accuracy_score(y_test , model.predict(X_test)))
    if (name == 'SVM linear'):
        roc_auc_score.append(metrics.roc_auc_score(y_test , model.decision_function(X_test)))        
    else:    
        roc_auc_score.append(metrics.roc_auc_score(y_test , model.predict_proba(X_test)[:,1]))

SVM linear
precision_score: 0.7341772151898734
recall_score: 0.6304347826086957
f1_score: 0.6783625730994152
accuracy_score: 0.7834645669291339
roc_auc_score: 0.8299785292538916
SVM rbf
precision_score: 0.7605633802816901
recall_score: 0.5869565217391305
f1_score: 0.6625766871165645
accuracy_score: 0.7834645669291339
roc_auc_score: 0.8349771873322598
LR
precision_score: 0.7215189873417721
recall_score: 0.6195652173913043
f1_score: 0.6666666666666666
accuracy_score: 0.7755905511811023
roc_auc_score: 0.8271604938271605
KNN
precision_score: 0.7352941176470589
recall_score: 0.5434782608695652
f1_score: 0.625
accuracy_score: 0.7637795275590551
roc_auc_score: 0.8052871712292001
DecisionTreeClassifier
precision_score: 0.703125
recall_score: 0.4891304347826087
f1_score: 0.576923076923077
accuracy_score: 0.7401574803149606
roc_auc_score: 0.7966317767042406
BaggingClassifier
precision_score: 0.6666666666666666
recall_score: 0.5434782608695652
f1_score: 0.5988023952095807
accuracy_score: 0.736220

In [51]:
import pandas as pd
d = {'precision_score': precision_score, 
     'recall_score': recall_score, 
     'f1_score': f1_score,
     'accuracy_score' : accuracy_score,
     'roc_auc_score' : roc_auc_score
    }
df = pd.DataFrame(data=d)
df.insert(loc=0, column='Method', value=['SVM linear','SVM rbf','LR','KNN', 'DecisionTreeClassifier','BaggingClassifier','RandomForestClassifier','ExtraTreesClassifier', 'AdaBoostClassifier','GradientBoostingClassifier','XGBClassifier', 'voting', 'MLP'])
df

,Method,precision_score,recall_score,f1_score,accuracy_score,roc_auc_score
0,SVM linear,0.734177,0.630435,0.678363,0.783465,0.829979
1,SVM rbf,0.760563,0.586957,0.662577,0.783465,0.834977
2,LR,0.721519,0.619565,0.666667,0.775591,0.827160
3,KNN,0.735294,0.543478,0.625000,0.763780,0.805287
4,DecisionTreeClassifier,0.703125,0.489130,0.576923,0.740157,0.796632
5,BaggingClassifier,0.666667,0.543478,0.598802,0.736220,0.818170
6,RandomForestClassifier,0.720588,0.532609,0.612500,0.755906,0.830784
7,ExtraTreesClassifier,0.750000,0.521739,0.615385,0.763780,0.834071
8,AdaBoostClassifier,0.651685,0.630435,0.640884,0.744094,0.813775
9,GradientBoostingClassifier,0.680000,0.554348,0.610778,0.744094,0.821826


# Wczytaj dane treningowe i testowe

In [53]:
# Wczytaj dane treningowe i testowe

import pandas as pd

train_set = pd.read_csv('Dane/adult/adult.data', sep=", ",header = None)
test_set = pd.read_csv('Dane/adult/adult.test', sep=", ",skiprows = 1, header = None) # Make sure to skip a row for the test set


col_labels = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 
              'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
             'wage_class']
train_set.columns = col_labels
test_set.columns = col_labels

train = train_set.replace('?', np.nan).dropna()
test = test_set.replace('?', np.nan).dropna()



dataset = pd.concat([train,test])

dataset['wage_class'] = dataset.wage_class.replace({'<=50K.': 0,'<=50K':0, '>50K.':1, '>50K':1})

dataset.drop(["fnlwgt"],axis=1,inplace=True)

dataset.drop(["education"],axis=1,inplace=True)

x = dataset.groupby('native_country')["wage_class"].mean()

d = dict(pd.cut(x[x.index!=" United-States"],5,labels=range(5)))

dataset['native_country'] = dataset['native_country'].replace(d)

dataset = pd.get_dummies(dataset,drop_first=True)

train = dataset.iloc[:train.shape[0]]
test = dataset.iloc[train.shape[0]:]

X_train = train.drop("wage_class",axis=1)
y_train = train.wage_class

X_test = test.drop("wage_class",axis=1)
y_test = test.wage_class

# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

# print(X_train.shape)
# X_test.shape

/var/folders/rl/pyfz32cs4rq01p8c57cwqj6m0000gn/T/ipykernel_14573/2503454130.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  train_set = pd.read_csv('Dane/adult/adult.data', sep=", ",header = None)
/var/folders/rl/pyfz32cs4rq01p8c57cwqj6m0000gn/T/ipykernel_14573/2503454130.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  test_set = pd.read_csv('Dane/adult/adult.test', sep=", ",skiprows = 1, header = None) # Make sure to skip a row for the test set


In [13]:
print(X_train.shape)
print(X_test.shape)

(30162, 41)
(15060, 41)


# Zad
Porównaj wyniki sieci na:
* oryginalnych danych 
* na wystandaryzowanych

In [54]:
#oryginalne

model = MLPClassifier((20,10))
model.fit(X_train, y_train)


from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, model.predict(X_test)))

0.8308764940239044


In [55]:
#skalowane

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

model = MLPClassifier((20,10))
model.fit(X_train, y_train)


from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, model.predict(X_test)))

0.8466135458167331


/Users/mateuszrajski/psi/podstawy_sztucznej_inteligencji/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
